In [ ]:
import os
import _pickle as pickle
import numpy as np
cstk_path = '/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Third_2019Mar22/codestacks/'
nrandom = 50
poses = [i for i in os.listdir(cstk_path) if os.path.isdir(os.path.join(cstk_path, i))]
#poses = pickle.load(open('/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Second_2019Feb17/results/unfinished_pos.pkl','rb'))
finished_pos = []
for pos in poses:
    try:
        processing = pickle.load(open(os.path.join(cstk_path,pos,'processing.pkl'),'rb'))
        if len(processing)==18:
            finished_pos.append(pos)
    except:
        continue
print(len(finished_pos))
if nrandom<len(finished_pos):
    subset = np.random.choice(finished_pos, size=nrandom, replace=False)
else:
    np.random.shuffle(poses)
    subset = poses
for i in range(len(subset)):
    pos = subset[i]
    processing = pickle.load(open(os.path.join(cstk_path,pos,'processing.pkl'),'rb'))
    new_pos = 'Empty'
    if len(processing) !=18:
        print(pos,' Isnt Finished Imaging')
        while (len(processing) !=18):
            new_pos = np.random.choice(poses, size=1, replace=False)
            if new_pos in subset:
                continue
            try:
                processing = pickle.load(open(os.path.join(cstk_path,new_pos,'processing.pkl'),'rb'))
            except:
                continue
    subset[i] = new_pos[0]

0
Pos68  Isnt Finished Imaging


/home/zach/miniconda3/envs/pyspots/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [26]:
finished_pos

[]

In [40]:
import os
import sys
import pickle
import argparse
import importlib
import traceback
import numpy as np
import pandas as pd
import dill as pickle
import multiprocessing
from skimage import io
from metadata import Metadata
from functools import partial
from fish_results import HybeData
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
from collections import defaultdict, Counter
from skimage.measure import regionprops, label
from scipy.spatial import distance_matrix,KDTree

cstk_path = '/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Second_2019Feb17/codestacks/'
pos = 'Pos2'
z = 29

seqfish_config = importlib.import_module('seqfish_config_Inflammation')
try:
    genes = seqfish_config.gids+seqfish_config.bids
except:
    genes = seqfish_config.gids
bitmap = seqfish_config.bitmap
normalized_gene_vectors = seqfish_config.norm_gene_codeword_vectors

hdata = HybeData(os.path.join(cstk_path,pos))
cstk = hdata.load_data(pos,z,'cstk')
cimg = hdata.load_data(pos,z,'cimg')
nf = hdata.load_data(pos,z,'nf')

class_img = cimg
cvectors = normalized_gene_vectors
zindex = z
pix_thresh=0
ave_thresh=0
spot_sum_thresh=2**12

#def parse_classification_image(class_img, cstk, cvectors, genes, zindex, pix_thresh=0, ave_thresh=0,spot_sum_thresh=2**12):
nf = np.zeros(cvectors.shape[1])
nf_dict = {}
for bit in range(cvectors.shape[1]):
    nf_dict[bit] = {}
    for gene in genes:
        nf_dict[bit][gene]=[]
label2d = label((class_img+1).astype('uint16'), neighbors=8)
properties = regionprops(label2d, (class_img+1).astype('uint16'))
areas = []
nclasses = []
multiclass_sets = 0
#bit_values = defaultdict(list)
gene_call_rows = []
below_threshold_rows = []
for prop in properties:
    coords = prop.coords
    centroid = prop.centroid
    classes = list(set(prop.intensity_image.flatten())-set([0]))
    #classes = list(set(list())-set([0]))
    if len(classes)==0:
        print('Label with no classes.', end='')
        pdb.set_trace()
        continue
    elif not len(classes)==1:
        #print('Labels need to be broken apart more than one classification found per label.', end='')
        #print(classes)
        pdb.set_trace()
        multiclass_sets+=1
        continue
    else:
        nclasses.append(len(classes))
        areas.append(prop.area)
    codeword_idx = classes[0]-1
    gene = genes[codeword_idx]
    bits = np.where(cvectors[codeword_idx]>0)[0]
    spot_pixel_values = []
    spot_pixel_means = []
    spot_sums = 0
    spot_nf = np.zeros(cvectors.shape[1])
    for b in bits:
        spot_bit_intensities = cstk[coords[:,0], coords[:,1], b]
        spot_nf[b] = np.mean(spot_bit_intensities)
        spot_pixel_values.append(spot_bit_intensities)
    spot_sum = np.sum(spot_pixel_values)
    spot_mean = np.mean(spot_pixel_values)

    if spot_sum > spot_sum_thresh:
        for b in bits:
            nf_dict[b][gene].append(spot_nf[b])

    if (len(coords)>pix_thresh) and (spot_mean>ave_thresh):
        gene_call_rows.append([genes[codeword_idx], spot_sum, centroid,
                        np.mean(spot_mean), len(coords), codeword_idx])
    else:
        below_threshold_rows.append([genes[codeword_idx], spot_sum, centroid,
                    np.mean(spot_mean), len(coords), codeword_idx])
df = pd.DataFrame(gene_call_rows, columns=['gene', 'ssum', 'centroid', 'ave', 'npixels', 'cword_idx'])
for b in nf_dict.keys():
    bit_nf = []
    for gene,gene_bit_spot_intensities in nf_dict[b].items():
        if len(gene_bit_spot_intensities)>0:
            bit_nf.append(np.mean(gene_bit_spot_intensities))
    nf[b] = np.mean(bit_nf)
#    return df,nf

print(nf)

[2130.97275487 1606.92121191  911.97921058 1340.13350828 1706.8280917
 1390.23613344 1947.91499384 2286.71807328 1870.37831481 2011.70408843
 2455.82161886 2104.47099689 1127.01109265 1142.89514417 1330.25346123
 2800.77124462 2387.72668868 3088.21435263]


In [37]:

gene

'Abhd2'

In [33]:
cstk_path = '/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Second_2019Feb17/codestacks/'
pos = 'Pos2'
z = 29

seqfish_config = importlib.import_module(args.cword_config)
try:
    genes = seqfish_config.gids+seqfish_config.bids
except:
    genes = seqfish_config.gids
bitmap = seqfish_config.bitmap
normalized_gene_vectors = seqfish_config.norm_gene_codeword_vectors
normalized_all_gene_vectors = seqfish_config.norm_all_codeword_vectors

hdata = HybeData(os.path.join(cstk_path,pos))
cstk = hdata.load_data(pos,z,'cstk')
cimg = hdata.load_data(pos,z,'cimg')
nf = hdata.load_data(pos,z,'nf')

df,nf = parse_classification_image(class_img, cstk, cvectors, genes, zindex, pix_thresh=0, ave_thresh=0,spot_sum_thresh=2**12)

In [6]:
import numpy as np
from fish_results import HybeData
import os
good_positions = pickle.load(open('/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Second_2019Feb17/results/tforms.pkl', 'rb'))['good']
out_path = '/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Second_2019Feb17/codestacks'
func_inputs = []
for p, t in good_positions.items():
    tforms_xyz = {k: (v[0][0], v[0][1], int(np.round(np.mean(v[0][2])))) for k, v in t.items()}
    txy = {k: (v[0], v[1]) for k, v in tforms_xyz.items()}
    tzz = {k: v[2] for k, v in tforms_xyz.items()}
    func_inputs.append((HybeData(os.path.join(out_path, p)), p, txy, tzz))

In [28]:
import pickle
spotcalls = pickle.load(open('/hybedata/Images/Zach/FISH_Troubleshooting/Predigest_WholeMount_First_2018Nov23-Corrected/results/spotcalls_reduced_size.pkl','rb'))

In [30]:
from analysis_scripts.iterative_classify_codestacks_and_parse import *
purged_df = purge_wrapper(spotcalls,4)

Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zach/miniconda3/envs/pyspots/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zach/miniconda3/envs/pyspots/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/zach/miniconda3/envs/pyspots/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zach/miniconda3/envs/pyspots/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zach/miniconda3/envs/pyspots/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zach/miniconda3/envs/pyspots/lib/python3.6/multiprocessing/process.py", line 93, in run
    se

KeyboardInterrupt: 

In [10]:
pos = 'Pos1'
t = good_positions[pos]
tforms_xyz = {k: (v[0][0], v[0][1], int(np.round(np.mean(v[0][2])))) for k, v in t.items()}
txy = {k: (v[0], v[1]) for k, v in tforms_xyz.items()}
tzz = {k: v[2] for k, v in tforms_xyz.items()}

In [11]:
txy

{'hybe1': (0, 0),
 'hybe2': (0.39999960162194115, -1.4018688058418503e-06),
 'hybe3': (4.59999741512415, -1.3999786018035056),
 'hybe4': (6.1999924120500465, 49.79999827682558),
 'hybe5': (1.8002094666964208, 43.599457450684724),
 'hybe6': (-5.399999998881384, 42.62840343323988),
 'hybe7': (-0.800006710664406, 43.600020509568324),
 'hybe8': (2.5999933563490365, 40.80002878635906),
 'hybe9': (1.3999958355248128, 46.1999939704726),
 'nucstain': (1.5999961860480942, 38.40000421142031)}

In [1]:
import pickle
tforms = pickle.load(open('/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Second_2019Feb17/results/tforms.pkl','rb'))
print(len(tforms['good'].keys()),len(tforms['bad'].keys()))

299 77


In [21]:
def classify_tform(tform_dict,pos,max_thresh):
    goodness = 0
    if type(tform_dict) != dict:
        #print('No beads in', pos)
        goodness = 1
    else:
        for hybe in tform_dict.keys():
            #print(tform_dict[hybe][1])
            if hybe != 'nucstain':
                if isinstance(tform_dict[hybe][0], str):
                    goodness = goodness + 1
                    #print(pos, hybe, 'not enough bead pairs found')
                elif tform_dict[hybe][1] > max_thresh:
                    goodness = goodness + 1
                    #print(pos, hybe, 'residual is too high',tform_dict[hybe][1])
    return goodness

In [26]:
max_thresh = 2
for pos in tforms['good'].keys():
    if classify_tform(tforms['good'][pos],pos,max_thresh)>0:
        print(pos)

In [13]:
import pickle
beads = pickle.load(open('/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Second_2019Feb17/results/beads.pkl','rb'))
print(len(beads.keys()))

355


In [7]:
from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree
import numpy as np
import sys
def purge_zoverlap(Input,z_dist = 2):
    posdf = Input['posdf']
    print('Starting',Input['posname'],':',len(posdf),'Total Spots')
    zidxes = posdf.z.unique()
    for z_i in range(len(zidxes)-1):
        subdf = posdf[(posdf.z==zidxes[z_i]) | (posdf.z==zidxes[z_i+1])]
        yx = subdf.centroid.values
        yx = np.stack(yx, axis=0)
        tree = KDTree(yx)
        dclust = DBSCAN(eps=2, min_samples=2)
        dclust.fit(yx)
        skip_list = set(np.where(dclust.labels_==-1)[0])
        nomatches = []
        drop_list = []
        for idx, i in enumerate(yx):
#             if idx % 10000 == 0:
#                 print(idx)
            if idx in skip_list:
                continue
            m = tree.query_ball_point(i, 2)
            m = [j for j in m if j!=idx]

            row_query = subdf.iloc[idx]
            for j in m:
                row_match = subdf.iloc[j]
                if row_match.cword_idx!=row_query.cword_idx:
                    continue

                if row_match.npixels>=row_query.npixels:
                    drop_list.append((idx, j))
                else:
                    drop_list.append((j, idx))
                    break
        if len(drop_list)>0:
            droppers, keepers = zip(*drop_list)
            posdf = posdf.drop(index=subdf.iloc[list(droppers)].index,inplace=True)
    return posdf

import pickle
#df = pickle.load(open('/hybedata/Images/Zach/FISH_Troubleshooting/Predigest_WholeMount_First_2018Nov23-Corrected/codestacks2/spotcalls.pkl','rb'))

import multiprocessing
import pandas
ncpu = 1
poses = df.posname.unique()
dfdic = {}
totaldf = pandas.DataFrame(columns = df.columns)
with multiprocessing.Pool(ncpu) as ppool:
    sys.stdout.flush()
    for result in ppool.imap(purge_zoverlap, [{'posname': pos, 'posdf': df[df.posname==pos]} for pos in poses]):
        totaldf = totaldf.append(result)
    ppool.close()
    sys.stdout.flush()

Starting 4-Pos_001_001 : 294890 Total Spots :type <class 'pandas.core.frame.DataFrame'>
[ 5  7  9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45 47 49 51
 53 55 57 59 61 63 65 67 69 71 73 75 77 79 81 83 85 87 89 91 93 95 97 99]
      gene     ssum                      centroid          ave  npixels  \
0   Il1rap   3519.0                  (0.0, 150.5)   439.875000        2   
1     Jak2   1335.0                  (0.0, 256.0)   333.750000        1   
2  Tnfaip3   1745.0                  (0.0, 396.0)   436.250000        1   
3     Plau   1401.0                  (0.0, 725.0)   350.250000        1   
4     Nav2   1341.0                  (1.0, 194.0)   335.250000        1   
5     Nav2   1667.0                  (2.0, 722.0)   416.750000        1   
6     Il7r   1696.0                 (10.0, 306.0)   424.000000        1   
7     Jaml   1375.0                (20.0, 1915.0)   343.750000        1   
8    Vps35  34569.0  (28.285714285714285, 1141.0)  1234.607143        7   
9    Ccnd1  811

AttributeError: 'NoneType' object has no attribute 'z'

In [ ]:
print(len(df))
print(len(totaldf))
print(100*(len(df)-len(totaldf))/len(df),'% spots removed')

In [ ]:
for pos in df.posname.unique():
    posdf = df[df.posname==pos]
    for z in posdf.z.unique():
        subdf = posdf[posdf.z==z]
        droppers = dfdic[pos][z]
        df.drop(index=subdf.iloc[list(droppers)].index,inplace=True)

In [ ]:
import multiprocessing
ncpu = 5
poses = df.posname.unique()
droppers = []
with multiprocessing.Pool(ncpu) as ppool:
    for result,pos in ppool.imap(purge_zoverlap, [{'posname': pos, 'posdf': df[df.posname==pos]} for pos in poses]):
        droppers.append(result)
        print(pos,' Finished')
for pos in dfdic.keys():
    for z in dfdic[pos].keys():
        droppers = dfdic[pos][z]
        df.drop(index=droppers,inplace=True)

In [29]:
drop_list = []
for pos in dfdic.keys():
    for z in dfdic[pos].keys():
        droppers = dfdic[pos][z]
        for spot in droppers:
            drop_list.append(spot)
purged_df = df.drop(index = list(dict.fromkeys(drop_list)),inplace=False)


In [25]:
print(len(drop_list))
print(len(list(dict.fromkeys(drop_list))))

3043992
42614


In [26]:
print(len(df))

4789407


In [30]:
print(len(purged_df))

4746793


In [28]:
list(dict.fromkeys(drop_list))

[11,
 12,
 4206,
 17,
 19,
 21,
 4212,
 4215,
 32,
 4221,
 39,
 41,
 4228,
 4229,
 4235,
 4236,
 52,
 53,
 59,
 60,
 4245,
 62,
 63,
 4249,
 65,
 4251,
 68,
 70,
 71,
 73,
 4260,
 4263,
 76,
 77,
 79,
 80,
 4279,
 83,
 4283,
 4284,
 90,
 4289,
 92,
 4300,
 4299,
 95,
 96,
 98,
 4307,
 4311,
 103,
 105,
 107,
 4317,
 4320,
 112,
 113,
 116,
 117,
 4348,
 121,
 122,
 125,
 128,
 4362,
 4357,
 4370,
 4378,
 4376,
 4387,
 142,
 4392,
 4394,
 145,
 146,
 147,
 148,
 151,
 153,
 4412,
 155,
 158,
 4453,
 4429,
 4434,
 164,
 165,
 166,
 4452,
 168,
 170,
 172,
 4468,
 176,
 181,
 182,
 4503,
 184,
 4504,
 187,
 189,
 190,
 191,
 4524,
 193,
 4517,
 4519,
 197,
 4536,
 199,
 200,
 201,
 4525,
 4529,
 205,
 4535,
 208,
 4557,
 213,
 214,
 215,
 4555,
 218,
 4586,
 220,
 221,
 222,
 223,
 4578,
 4590,
 226,
 228,
 4595,
 4580,
 234,
 4603,
 236,
 4607,
 242,
 4617,
 244,
 245,
 4619,
 247,
 248,
 249,
 250,
 251,
 252,
 257,
 4661,
 261,
 4647,
 263,
 264,
 4654,
 266,
 269,
 270,
 4663,
 4671,
